In [2]:
!pip install trafilatura

Defaulting to user installation because normal site-packages is not writeable
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/132.6 kB ? eta -:--:--
   ---------------------------------------- 132.6/132.6 kB 8.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/101.8 kB ? eta -:--:--
   ---------------------------------------- 101.8/101.8 kB 6.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/837.8 kB ? eta -:--:--
   -------------------- ------------------ 450.6/837.8 kB 13.8 MB/s eta 0:00:01
   --------------------------------------- 837.8/837.8 kB 10.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------- -------------------------------- 0.7/3.8 MB 14.2 MB/s eta 0:00:01
   ---------------- ----------------------- 1.6/3.8 MB 16.9 MB/s eta 0:00:01
   ---------------------------- ----------- 2.7/3.8 MB 18.9 MB/s eta 0:00:01
   ----

DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2023.12.0 requires fsspec==2023.12.0, but you have fsspec 2023.10.0 which is incompatible.
pinecone-datasets 0.5.0rc11 requires pyarrow<12.0.0,>=11.0.0, but you have pyarrow 15.0.0 which is incompatible.
pinecone-datasets 0.5.0rc11 requires pydantic<2.0.0,>=1.10.5, but you have pydantic 2.10.2 which is incompatible.
s3fs 2023.12.0 requires fsspec==2023.12.0, but you have fsspec 2023.10.0 which is incompatible.
stix2-validator 3.1.4 requires jsonschema[format-nongpl]<4.18.0,>=4.6.0, but you

In [ ]:
import trafilatura
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Read URLs from the file
urls = open('sources/urls_web.txt').read().split()

# Function to process a single URL
def process_url(url):
    try:
        content = trafilatura.fetch_url(url)
        if content:
            extracted = trafilatura.extract(content, output_format='json', include_comments=False)
            if extracted:
                extracted_json = json.loads(extracted)  # Parse the JSON string
                text = extracted_json.get("text", "")  # Get only the "text" field
                return {"url": url, "Text": text}
    except Exception as e:
        return {"url": url, "Text": f"Error: {str(e)}"}

    return {"url": url, "Text": ""}  # Default if nothing was fetched or extracted

# Use ThreadPoolExecutor for multithreading
results = []
with ThreadPoolExecutor() as executor:
    # Submit all URLs to the executor
    futures = {executor.submit(process_url, url): url for url in urls}

    # Use tqdm to display progress
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing URLs"):
        results.append(future.result())

# Save the results to a JSON file
with open('output.json', 'w', encoding='utf-8') as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=4)

print("Data has been saved to output.json.")


Processing URLs:   1%|▏         | 1989/151806 [02:18<2:53:36, 14.38it/s]
